In [2]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [3]:
!pip install PyPDF2 langchain langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.2 MB/s eta 0:00:00


In [4]:
!  pip install PyPDF2

In [5]:
!pip install  langchain langchain_community langchain_groq PyPDF2 concurrent

ERROR: Could not find a version that satisfies the requirement concurrent (from versions: none)
ERROR: No matching distribution found for concurrent


In [18]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 32.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 77.7 MB/s eta 0:00:00:00:0100:01


In [26]:
import os
import tempfile
from pydantic import BaseModel
from langchain_community.document_loaders import TextLoader, PyMuPDFLoader
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

# class DocumentClassificationResponse(BaseModel):
#     """
#     A Pydantic model to structure the response for document classification and restoration.
#     """
#     file_type: str
#     restored_content: str

# class DocumentSummarizationResponse(BaseModel):
#     """
#     A Pydantic model to structure the response for document summarization.
#     """
#     file_type: str
#     summary: str

class DocumentProcessor:
    """
    A class to handle document loading, classification, restoration, and summarization using language models.
    """

    def __init__(self):
        """
        Initializes the DocumentProcessor with necessary configurations.
        """
        self.llm = ChatGroq(model_name="Llama3-8b-8192", api_key="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")
        self.classification_template = PromptTemplate.from_template(self._get_classification_template())
        self.restoration_template = PromptTemplate.from_template(self._get_restoration_template())
        self.summary_template = PromptTemplate.from_template(self._get_summary_template())
        self.classification_chain = LLMChain(llm=self.llm, prompt=self.classification_template)
        self.restoration_chain = LLMChain(llm=self.llm, prompt=self.restoration_template)
        self.summary_chain = LLMChain(llm=self.llm, prompt=self.summary_template)

    @staticmethod
    def _get_classification_template():
        """
        Returns the classification template for the LLM.

        Returns:
            str: The template string for document classification.
        """
        return """<s>[INST] The following is an article:
        {text}
        Based on this, please classify the document into one of the following categories: Civil, Criminal, Writ.
        You can use below paragraph for classification of the document: 
        "Civil cases involve disputes between individuals, organizations, or the government, seeking legal remedies like compensation for issues such as contract disputes, torts, family law matters, property disputes, and employment issues.
        Criminal cases are offenses against the state or public, with the government prosecuting individuals or entities for crimes like felonies (murder, robbery), misdemeanors (petty theft), and infractions (traffic violations), aiming to punish and deter criminal behavior. 
        Writ cases involve judicial orders to correct legal errors or enforce rights, with common writs including habeas corpus (ensuring lawful detention), mandamus (ordering a public official to perform a duty), prohibition (stopping a lower court from exceeding its jurisdiction), certiorari (reviewing a lower court's decision), and quo warranto (challenging the right to hold public office)."
        Answer with the category only and nothing else:  [/INST] </s>"""

    @staticmethod
    def _get_restoration_template():
        """
        Returns the restoration template for the LLM.

        Returns:
            str: The template string for document restoration.
        """
        return """<s>[INST] The following is an article:
        {text}
        Based on this, please restore the content of the document by removing anomalous characters and gibberish to produce comprehensible content with proper grammar. 
        Answer:  [/INST] </s>"""

    @staticmethod
    def _get_summary_template():
        """
        Returns the summary template for the LLM.

        Returns:
            str: The template string for document summarization.
        """
        return """<s>[INST] The following is an article:
        {text}
        Based on this, please provide a summary. 
        Answer:  [/INST] </s>"""

    def load_documents(self, file_path):
        """
        Loads documents from the given file path.

        Args:
            file_path (str): The path to the file to be loaded.

        Returns:
            list: A list of document objects.

        Raises:
            ValueError: If the file format is unsupported.
        """
        if file_path.lower().endswith('.pdf'):
            loader = PyMuPDFLoader(file_path)
        elif file_path.lower().endswith('.txt'):
            loader = TextLoader(file_path)
        else:
            raise ValueError("Unsupported file format. Please provide a .pdf or .txt file.")
        return loader.load()

    def classify_and_restore_file(self, file_path: str):
        """
        Classifies and restores the content of the document from the given file path.

        Args:
            file_path (str): The path to the file to be processed.

        Returns:
            dict: A dictionary containing the file type and restored content.
        """
        docs = self.load_documents(file_path)
        full_text = " ".join([doc.page_content for doc in docs])

        file_type = self.classification_chain.run({"text": full_text}).strip()
        restored_content = self.restoration_chain.run({"text": full_text})

        return {"file_type": file_type, "restored_content": restored_content}

    def classify_and_summarize_file(self, file_path: str):
        """
        Classifies and summarizes the content of the document from the given file path.

        Args:
            file_path (str): The path to the file to be processed.

        Returns:
            dict: A dictionary containing the file type and summary.
        """
        docs = self.load_documents(file_path)
        full_text = " ".join([doc.page_content for doc in docs])

        file_type = self.classification_chain.run({"text": full_text}).strip()
        summary = self.summary_chain.run({"text": full_text})

        return {"file_type": file_type, "summary": summary}


# document_processor = DocumentProcessor()
# document_processor.classify_and_summarize_file(r"C:\Users\ayush\OneDrive\Desktop\Cm_work\cm_om\Amar_Singh_Thukral_S_O_Shri_Joumphi_vs_Sandeep_Chhatwal_S_O_Shri_Peshori_Lal_on_5_July_2004.PDF")


def main():
    # Create an instance of DocumentProcessor
    document_processor = DocumentProcessor()
    
    # Path to the document you want to test
    file_path = r"/kaggle/input/criminal-6072/CRA_6072_2024_MEMO OF APPEAL_2024-05-16_9402965.pdf"  # Change this to the actual path of your file

    # Call classify_and_restore_file method
    try:
        classification_and_restoration_result = document_processor.classify_and_restore_file(file_path)
        #print("Classification and Restoration Result:")
        #print("File Type:", classification_and_restoration_result['file_type'])            
        restored_content=classification_and_restoration_result['restored_content']
        print("Restored Content:",restored_content)
    except Exception as e:
        print(f"Error during classification and restoration: {e}")
    return restored_content

if __name__ == "__main__":
    restored_content=main()

Restored Content: Here is the restored content of the document:

**IN THE HIGH COURT OF MADHYA PRADESH**

**PRINCIPAL SEAT AT JABALPUR**

**Criminal Appeal No. 672/2024**

**APPELLANT**

**Sohel Kapoor**

**(In Jail)**

**S/o Shri Saleem Kapoor, aged about 29 years, Occupation Labour, R/o Ward no.13, Near CMO Bangla, Patel Ward Multai, District Betul (M.P.)**

**RESPONDENTS**

**1. The State of Madhya Pradesh**
**Through Police Station Kotwali-Betul, District Betul (M.P.)**
**2. Victim-A**

**APPEAL UNDER SECTION 14(A) OF SC/ST (PREVENTION OF ATROCITIES) ACT 1989 FOR GRANT OF BAIL**

**Whether any bail application is pending before or already disposed of?**

**Institution No. Date of Institution Result**

* Hon'ble Supreme Court of India: Nil
* Hon'ble High Court: Nil
* Sub Court: Nil
* B.A. No. 14-05-2024: Rejected
* Ordinate to High Court: 200304/2024

**PARTICULARS OF CRIME**

**Crime No. 151/2024**
**Police Station— Kotwali Betul, District Betul (M.P.)**
**Offence: U/s 294, 323, 50

In [40]:
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from concurrent.futures import ThreadPoolExecutor
from tenacity import retry, stop_after_attempt, wait_exponential
import re
from langchain_groq import ChatGroq
import ast
import json


model_name = "llama3-8b-8192"
llm = ChatGroq(temperature=0, groq_api_key="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", model=model_name)

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def split_text(text, chunk_size=10000, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    return text_splitter.split_text(text)

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def extract_relations(chunk, llm):
    try:
        prompt = f"""
You are an expert in analyzing Indian criminal case documents. Extract the following entities from the text:

- CASE_NO
- COURT
- JUDGE
- DATE
- LAWYER
- PETITIONER
- RESPONDENT
- WITNESS
- VICTIM
- CRIME
- SENTENCE
- CHARGE
- EVIDENCE
- POLICE
- FIR_NUMBER
- BAIL
- CRIME_SCENE
- FORENSIC_EVIDENCE
- JUDGEMENT

Afterwards, a detailed relation extraction on all entities recognised to output tuples as (entity,relation,entity).

Identify the relationships between the extracted entity value and output a list of dictionaries containing head,relation,tail.Further head and tails will contain entity name and entity type each.
The tuples you generate will be used to develop a knowledge graph in accordance to the relations between entities later.

Text: {chunk}

OUTPUT STRUCTURE-
{{"head": {{"name": entity1_name, "type": ENTITY1_TYPE}}, "relation": RELATION_NAME, "tail": {{"name": entity2_name, "type": ENTITY2_TYPE}}}}

INSTRUCTIONS-
-no tuples should be repeated.
-try to connect CASE_NO to all entities with relations that are good in a legal and professional manner  
-revise the tuples before outputand only keep important and clear tuples which have valid values
-entities which are none,na or not defined should not be included.
-remove double quotes from the entities and relations in tuples. ("'COURT'" -> 'COURT')
-entity "type" and "relation" values should be uppercase

THE BELOW EXAMPLE is NOT TO BE INCLUDED IN YOUR RESPONSE AS IT JUST FOR REFERENCE AND LEARNING
AN EXAMPLE OUTPUT DICTIONARY OF A RANDOM CRIMINAL FILE AND THE FORMAT OF THIS SHOULD BE IMITATED -

[
{{HEAD: {{NAME: "Case No. 1234/2024", TYPE: "CASE_NO"}}, RELATION: "BELONGS_TO", TAIL: {{NAME: "Judge XYZ", TYPE: "JUDGE"}}}},
{{HEAD: {{NAME: "District Court, ABC City", TYPE: "COURT"}}, RELATION: "HAS_JUDGE", TAIL: {{NAME: "Judge XYZ", TYPE: "JUDGE"}}}},
{{HEAD: {{NAME: "Judge XYZ", TYPE: "JUDGE"}}, RELATION: "PRESIDED_OVER", TAIL: {{NAME: "Case No. 1234/2024", TYPE: "CASE_NO"}}}},
{{HEAD: {{NAME: "Case No. 1234/2024", TYPE: "CASE_NO"}}, RELATION: "HEARD_ON", TAIL: {{NAME: "2024-07-16", TYPE: "DATE"}}}},
{{HEAD: {{NAME: "Lawyer ABC", TYPE: "LAWYER"}}, RELATION: "REPRESENTS", TAIL: {{NAME: "Accused XYZ", TYPE: "RESPONDENT"}}}},
{{HEAD: {{NAME: "Victim PQR", TYPE: "VICTIM"}}, RELATION: "AFFECTED_BY", TAIL: {{NAME: "Crime ABC", TYPE: "CRIME"}}}},
{{HEAD: {{NAME: "Crime ABC", TYPE: "CRIME"}}, RELATION: "COMMITTED_IN", TAIL: {{NAME: "Location XYZ", TYPE: "CRIME_SCENE"}}}},
{{HEAD: {{NAME: "Accused XYZ", TYPE: "RESPONDENT"}}, RELATION: "SENTENCED_TO", TAIL: {{NAME: "10 years imprisonment", TYPE: "SENTENCE"}}}},
{{HEAD: {{NAME: "Accused XYZ", TYPE: "RESPONDENT"}}, RELATION: "CHARGED_WITH", TAIL: {{NAME: "Murder", TYPE: "CHARGE"}}}},
{{HEAD: {{NAME: "Forensic Report ABC", TYPE: "EVIDENCE"}}, RELATION: "RELATED_TO", TAIL: {{NAME: "Crime ABC", TYPE: "CRIME"}}}},
{{HEAD: {{NAME: "Officer XYZ", TYPE: "POLICE"}}, RELATION: "INVESTIGATED", TAIL: {{NAME: "Case No. 1234/2024", TYPE: "CASE_NO"}}}},
{{HEAD: {{NAME: "FIR No. 5678", TYPE: "FIR_NUMBER"}}, RELATION: "FILED_FOR", TAIL: {{NAME: "Crime ABC", TYPE: "CRIME"}}}},
{{HEAD: {{NAME: "Accused XYZ", TYPE: "RESPONDENT"}}, RELATION: "GRANTED_BAIL", TAIL: {{NAME: "Bail Granted", TYPE: "BAIL_STATUS"}}}},
{{HEAD: {{NAME: "Witness ABC", TYPE: "WITNESS"}}, RELATION: "TESTIFIED_IN", TAIL: {{NAME: "Case No. 1234/2024", TYPE: "CASE_NO"}}}},
{{HEAD: {{NAME: "Location XYZ", TYPE: "CRIME_SCENE"}}, RELATION: "RELATED_TO", TAIL: {{NAME: "Crime ABC", TYPE: "CRIME"}}}},
{{HEAD: {{NAME: "Blood Sample XYZ", TYPE: "FORENSIC_EVIDENCE"}}, RELATION: "COLLECTED_FROM", TAIL: {{NAME: "Crime Scene ABC", TYPE: "CRIME_SCENE"}}}},
{{HEAD: {{NAME: "Judge XYZ", TYPE: "JUDGE"}}, RELATION: "ISSUED", TAIL: {{NAME: "Guilty Verdict", TYPE: "JUDGMENT"}}}}
]

-the ouput you produce should strictly just be a list of dictionary of tuples and no other text or explanation should be generated
    """
        response = llm.predict(prompt)
        return response
    except Exception as e:
        print(f"Error processing chunk: {e}")
        raise

def process_chunks_batch(chunks, llm, batch_size=1000):
    all_relations = []
    with ThreadPoolExecutor() as executor:
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i:i+batch_size]
            results = list(executor.map(lambda chunk: extract_relations(chunk, llm), batch))
            all_relations.extend(results)
    return all_relations

def combine_results(all_relations):
    return "\n".join(all_relations)

def extract_dictionaries(text):
    dictionaries = []
    current_dict = []
    in_dict = False
    bracket_depth = 0

    i = 0
    while i < len(text):
        char = text[i]

        if char == '{':
            bracket_depth += 1
            if bracket_depth == 1:
                current_dict = []
                in_dict = True
            current_dict.append(char)

        elif char == '}':
            current_dict.append(char)
            bracket_depth -= 1
            if bracket_depth == 0 and in_dict:
                in_dict = False
                try:
                    parsed_dict = ast.literal_eval("".join(current_dict))
                    if isinstance(parsed_dict, dict):
                        dictionaries.append(parsed_dict)
                except:
                    pass
                current_dict = []

        elif in_dict:
            current_dict.append(char)

        i += 1

    return dictionaries

#helper function for remove_duplicate_dicts
def dict_to_tuple(d):
    if isinstance(d, dict):
        return tuple((k, dict_to_tuple(v)) for k, v in sorted(d.items()))
    elif isinstance(d, list):
        return tuple(dict_to_tuple(v) for v in d)
    else:
        return d

def remove_duplicate_dicts(dict_list):
    seen = set()
    unique_dicts = []
    for d in dict_list:
        # Convert the dictionary to a tuple representation
        dict_tuple = dict_to_tuple(d)
        if dict_tuple not in seen:
            seen.add(dict_tuple)
            unique_dicts.append(d)
    return unique_dicts


@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def disambiguate_entities(tuples, llm):
    try:
        prompt = f"""
you are a legal expert who is very good at entity disambiguation.
You are provided with a dictionary of tuples representing entities and their relationships in the following format:
[
    "head": {{ 
        "name": "Entity Name",
        "type": "Entity Type"
    }},
    "relation": "Relationship Type",
    "tail": {{
        "name": "Entity Name",
        "type": "Entity Type"
    }}
]
Your task is to manually perform entity disambiguation on data by handling entities that are similar or have minor spelling differences. Additionally, ensure that the entity name and entity type are not the same (e.g., "name": "Government" and "type": "government" should be corrected).
Identify entities with similar names or minor spelling differences (e.g., "court", "Court", "court'", "COURT") and combine them into a single entity.
Correct any entities where the name and type are identical or very similar, ensuring that the type accurately represents the entity category.
Return the corrected dictionary with disambiguated entities and relationships.
Example Input:
[
    {{"head": {{"name": "High Court of Madhya Pradesh", "type": "court"}}, "relation": "has_registered", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "case_number"}}}},
    {{"head": {{"name": "High Court of Madhya Pradesh", "type": "court"}}, "relation": "decided", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "case_number"}}}},
    {{"head": {{"name": "High Court of MP", "type": "court"}}, "relation": "reviewed", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "case_number"}}}},
    {{"head": {{"name": "court", "type": "court"}}, "relation": "has_case", "tail": {{"name": "case_number", "type": "case_number"}}}},
]
Expected Output:
[   {{"head": {{"name": "High Court of Madhya Pradesh", "type": "court"}}, "relation": "has_registered", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "case_number"}}}},
    {{"head": {{"name": "High Court of Madhya Pradesh", "type": "court"}}, "relation": "decided", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "case_number"}}}},
    {{"head": {{"name": "High Court of Madhya Pradesh", "type": "court"}}, "relation": "reviewed", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "case_number"}}}},
    {{"head": {{"name": "High Court of Madhya Pradesh", "type": "court"}}, "relation": "has_case", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "case_number"}}}},
]
Instructions:
try to retain the size of tuples and dont lose entities
Ensure for each entity, its name and type are not the same.
Return the disambiguated and corrected dictionary in the given format
your response should just contain the output data structure

Actual input- {tuples} 
        """
        response = llm.predict(prompt)
        return response
    except Exception as e:
        print(f"Error disambiguating entities: {e}")
        raise

def get_relation_tuples_from_pdf(pdf_path):
    content = extract_text_from_pdf(pdf_path)
    chunks = split_text(content)
    all_relations = process_chunks_batch(chunks, llm)
    final_result = combine_results(all_relations)
    dict_of_tuples = extract_dictionaries(final_result)
    unique_extracted_dictionaries = remove_duplicate_dicts(dict_of_tuples)
    #disambiguated_dict = disambiguate_entities(unique_extracted_dictionaries, llm)
    #extracted_dictionaries = extract_dictionaries(disambiguated_dict)
    return unique_extracted_dictionaries

def main(pdf_path):
    tuples=get_relation_tuples_from_pdf(pdf_path)
    return tuples 

# Example usage:
pdf_path = '/kaggle/input/aabbcc/CRA_262_2002_FinalOrder_30-Jun-2021.pdf'
response = main(pdf_path)

In [41]:
print(len(response),response)

43 [{'head': {'name': 'Cr. Appeal No.262/2002', 'type': 'CASE_NO'}, 'relation': 'BELONGS_TO', 'tail': {'name': 'HIGH COURT OF MADHYA PRADESH, PRINCIPAL SEAT', 'type': 'COURT'}}, {'head': {'name': 'HIGH COURT OF MADHYA PRADESH, PRINCIPAL SEAT', 'type': 'COURT'}, 'relation': 'HEARD_BY', 'tail': {'name': 'Justice Prakash Shrivastava', 'type': 'JUDGE'}}, {'head': {'name': 'Justice Prakash Shrivastava', 'type': 'JUDGE'}, 'relation': 'DELIVERED_JUDGMENT', 'tail': {'name': 'Cr. Appeal No.262/2002', 'type': 'CASE_NO'}}, {'head': {'name': 'Karan Singh', 'type': 'PETITIONER'}, 'relation': 'FILED_APPEAL', 'tail': {'name': 'Cr. Appeal No.262/2002', 'type': 'CASE_NO'}}, {'head': {'name': 'State of M.P.', 'type': 'RESPONDENT'}, 'relation': 'RESPONDED_TO', 'tail': {'name': 'Cr. Appeal No.262/2002', 'type': 'CASE_NO'}}, {'head': {'name': 'Justice Prakash Shrivastava', 'type': 'JUDGE'}, 'relation': 'DELIVERED_OPINION', 'tail': {'name': 'Cr. Appeal No.262/2002', 'type': 'CASE_NO'}}, {'head': {'name': 'K

In [8]:
!pip install neo4j


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 5.8 MB/s eta 0:00:00a 0:00:01


In [9]:
from neo4j import GraphDatabase

# Replace these with your AuraDB credentials
uri = "neo4j+s://6cb5ec24.databases.neo4j.io"
user = "neo4j"
password = "Ww3XWwg1qPUkKzeCoxmBGeM0Jw9pY3HZLQ2V7vzQwjA"



In [10]:
pip install py2neo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 3.8 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 10.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [42]:
from neo4j import GraphDatabase
import logging

# Create a connection to AuraDB
def create_auradb_connection(uri, user, password):
    try:
        driver = GraphDatabase.driver(uri, auth=(user, password))
        return driver
    except Exception as e:
        logging.error(f"Error creating AuraDB connection: {e}")
        raise

# Function to insert a tuple into AuraDB
def insert_tuple(tx, entity1_value, entity1_type, relation, entity2_value, entity2_type):
    try:
        query = f"""
                    MERGE (a:{entity1_type} {{name: $entity1_value}})
                    MERGE (b:{entity2_type} {{name: $entity2_value}})
                    MERGE (a)-[r:{relation}]->(b)
                """
        tx.run(query, entity1_value=entity1_value, entity1_type=entity1_type, 
               relation=relation, entity2_value=entity2_value, entity2_type=entity2_type)
    except Exception as e:
        logging.error(f"Error inserting tuple: {e}")
        raise

# Process and insert dictionaries into AuraDB
def process_and_insert_dicts(uri, user, password, dicts):
    driver = create_auradb_connection(uri, user, password)
    try:
        with driver.session() as session:
            for d in dicts:
                head = d['head']
                tail = d['tail']
                try:
                    session.write_transaction(
                        insert_tuple,
                        head['name'], head['type'],
                        d['relation'],
                        tail['name'], tail['type']
                    )
                except Exception as e:
                    logging.error(f"Error processing dictionary: {d}, Error: {e}")
                    continue
    except Exception as e:
        logging.error(f"Error during session operation: {e}")
        raise
    finally:
        driver.close()

# Process and insert dictionaries into AuraDB
process_and_insert_dicts(uri, user, password, response)


/tmp/ipykernel_34/1735363727.py:36: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(


In [43]:
#RENDERING EXISTING KNOWLEDGE GRAPH FROM AURADB AS HTML(WITH COLOUR)
from py2neo import Graph
from pyvis.network import Network
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random


driver = GraphDatabase.driver(uri, auth=(user, password))
graph = Graph(uri, auth=(user, password))

# Define a Cypher query to retrieve nodes and relationships
query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 400
"""

result = graph.run(query)

# Create a pyvis network
net = Network(height='1000px', width='100%', notebook=True, bgcolor='#FAF9F6', font_color='black')

# Function to generate a random color
def random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Dictionary to store colors for each label
label_colors = {}

# Add nodes and edges from the query result
for record in result:
    n = record["n"]
    r = record["r"]
    m = record["m"]

    # Ensure names and labels are strings for serialization
    n_name = str(n["name"])
    m_name = str(m["name"])
    n_labels = list(n.labels)
    m_labels = list(m.labels)
    r_type = r.__class__.__name__
    # Assign colors to labels dynamically
    if n_labels:
        for label in n_labels:
            if label not in label_colors:
                label_colors[label] = random_color()
        n_color = label_colors[n_labels[0]]
    else:
        n_color = random_color()

    if m_labels:
        for label in m_labels:
            if label not in label_colors:
                label_colors[label] = random_color()
        m_color = label_colors[m_labels[0]]
    else:
        m_color = random_color()

    net.add_node(n_name, label=n_name, title=", ".join(n_labels), color=n_color)
    net.add_node(m_name, label=m_name, title=", ".join(m_labels), color=m_color)
    net.add_edge(n_name, m_name,title=r_type, color='#FFD700')

# Generate network with options
net.show_buttons(filter_=['physics'])
net.show('knowledge_graph_not_restored.html')


knowledge_graph_not_restored.html
